# Aerospike Java Client – Reading and Updating Lists

This notebook demonstrates Java Aerospike CRUD operations (Create, Read, Update, Delete) for lists of data, focusing on server-side **read** and **update** operations, including **sort**. 

This [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/examples_index.html) requires the Aerospike database running locally with Java kernel and Aerospike Java Client. To create a Docker container that satisfies the requirements and holds a copy of these notebooks, visit the [Aerospike Notebooks Repo](https://github.com/aerospike-examples/interactive-notebooks).

## Notebook Setup 

Run these first to initialize Jupyter, download the Java Client, and make sure the Aerospike Database is running.

### Import Jupyter Java Integration 

Make it easier to work with Java in Jupyter.

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;

IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Start Aerospike

Ensure Aerospike database is running locally.

In [2]:
%sh asd

### Download the Aerospike Java Client

Ask Maven to download and install the project object model (POM) of the Aerospike Java Client.

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

### Start the Aerospike Java Client and Connect

Create an instance of the Aerospike Java Client, and connect to the demo cluster.

The default cluster location for the Docker container is *localhost* port *3000*. If your cluster is not running on your local machine, modify *localhost* and *3000* to the values for your Aerospike cluster.

In [4]:
import com.aerospike.client.AerospikeClient;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


## CREATING Lists in Aerospike

### Create and Print List Data

Create and print a String list and an Integer List. 

In [5]:
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import java.util.Map;

ArrayList<String> listStr = new ArrayList<String>();
listStr.add("Annette");
listStr.add("Redwood");
listStr.add("Aquamarine");
listStr.add("Pineapple");
System.out.println("String List: " + listStr);

ArrayList<Integer> listInt = new ArrayList<Integer>();
listInt.add(81);
listInt.add(3);
listInt.add(27);
listInt.add(9);
listInt.add(27);
listInt.add(1);
System.out.println("Integer List: " + listInt);

String List: [Annette, Redwood, Aquamarine, Pineapple]
Integer List: [81, 3, 27, 9, 27, 1]


### Insert the Lists into Aerospike

#### Create a Key Java Object

A **Key** identifies a specific record in your Aerospike server or cluster. Each key must have a **Namespace** and optionally a **Set** name. 
* A **Namespace** is like a database, in Aerospike.  
* A **Set** is like a database table in Aerospike. 

The namespace *test* is configured on your Aerospike server or cluster. Go [here](https://www.aerospike.com/docs/architecture/data-model.html) for additional information on the [Aerospike Data Model](https://www.aerospike.com/docs/architecture/data-model.html). 

In [6]:
import com.aerospike.client.Key;

String listSet = "listset1";
String listNamespace = "test";

Integer theKey = 0;

Key key = new Key(listNamespace, listSet, theKey);
System.out.println("Key created." );

Key created.


#### Create a Bin Java Object for Each List

A **Bin** is a data field in an Aerospike record.

In [7]:
import com.aerospike.client.Bin;

String listStrBin = "liststrbin";
String listIntBin = "listintbin";

Bin bin1 = new Bin(listStrBin, listStr);
Bin bin2 = new Bin(listIntBin, listInt);
System.out.println( "Created " + bin1 + " and " + bin2 + ".");

Created liststrbin:[Annette, Redwood, Aquamarine, Pineapple] and listintbin:[81, 3, 27, 9, 27, 1].


#### Create a Write Policy Java Object for Record Insertion 

A **Policy** tells Aerospike the intent of a database operation. 
Go [here](https://www.aerospike.com/docs/guide/policies.html) For more information on [policies](https://www.aerospike.com/docs/guide/policies.html).

In [8]:
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;

ClientPolicy clientPolicy = new ClientPolicy();
System.out.println("Created a client policy.");

Created a client policy.


#### Put the List Data into Aerospike

In [9]:
client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
System.out.println("Key: " + theKey + ", liststrbin: " + listStr + ", listintbin: " + listInt );

Key: 0, liststrbin: [Annette, Redwood, Aquamarine, Pineapple], listintbin: [81, 3, 27, 9, 27, 1]


## READING Lists and List Metadata From the Server

Now that the lists are in Aerospike, the client can return full or partial lists and even metadata about individual **bin** contents.

### Get the Record

A record can be retrieved using the **key**, **namespace**, and **set** name.

In the output: 
* **gen** is the generation number, the number of record writes. 
* **exp** is the expiration counter, the TTL for the record.
For more information on [both](https://www.aerospike.com/docs/guide/FAQ.html), see the [Aerospike FAQ](https://www.aerospike.com/docs/guide/FAQ.html).

In [10]:
import com.aerospike.client.Record;

Key key = new Key(listNamespace, listSet, theKey);
Record record = client.get(null, key);
System.out.println(record);

(gen:3),(exp:351830345),(bins:(liststrbin:[Annette, Redwood, Aquamarine, Pineapple]),(listintbin:[81, 3, 27, 9, 27, 1]))


### Get String Elements By Index and Rank

Get string list elements from the Aerospike Server or Cluster using index and rank. No data is modified by these ops.

#### Get the Last String

Aerospike provides operations to read list element(s) by **index**. As a convenience, the client returns the specified value as the contents of the bin.

Aerospike operations allow working forward from the beginning of the list using zero-based numbering. Negative numbers index backwards from the end of a list. For examples of [indexes](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html). 

In [11]:
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.ListOperation;

int last = -1;

Key key = new Key(listNamespace, listSet, theKey);
Record record = client.get(null, key);
Record lastString = client.operate(null, key, 
    ListOperation.get(listStrBin, last)
    );
    
System.out.println("The string list: " + record.getValue(listStrBin));
System.out.println("The last string: " + lastString.getValue(listStrBin));

The string list: [Annette, Redwood, Aquamarine, Pineapple]
The last string: Pineapple


#### Get the Highest Rank Item

Aerospike provides operations to read list item(s) by rank. For information on [list ranking](https://en.wikipedia.org/wiki/List_ranking), go [here](https://en.wikipedia.org/wiki/List_ranking).

Reading a record from Aerospike allows choice of what value to return. For the list of [options](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListReturnType.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListReturnType.html). 

In [12]:
import com.aerospike.client.cdt.ListReturnType;

int highestRank = -1;

Key key = new Key(listNamespace, listSet, theKey);
Record record = client.get(null, key);
Record highestRankString = client.operate(null, key, 
    ListOperation.getByRank(listStrBin, highestRank, ListReturnType.VALUE)
    );
System.out.println("The string list: " + record.getValue(listStrBin));
System.out.println("The highest rank string: " + highestRankString.getValue(listStrBin));

The string list: [Annette, Redwood, Aquamarine, Pineapple]
The highest rank string: Redwood


### Pull Integer Elements Using List Metadata

Read integer list metadata from the Aerospike Server or Cluster. No data is modified by these ops.

#### Pull Integers Between 3 and 27

In addition to reading list elements by rank and order, Aerospike provides operations to return a **Range** of elements by value.
* The lower bound of a range is included.
* The upper bound of a range is excluded.

For more examples of [ranges](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html). 

In [13]:
int lowerBound = 3;
int upperBound = 27;

Key key = new Key(listNamespace, listSet, theKey);
Record record = client.get(null, key);
Record between3And27 = client.operate(null, key, 
    ListOperation.getByValueRange(listIntBin, Value.get(lowerBound), Value.get(upperBound), ListReturnType.VALUE)
    );

System.out.println("The integer list: " + record.getValue(listIntBin));
System.out.println("The integers between " + lowerBound + " and " + upperBound + ": " + between3And27.getValue(listIntBin));

The integer list: [81, 3, 27, 9, 27, 1]
The integers between 3 and 27: [3, 9]


#### Pull the 2nd and 3rd Ranked Integers

Aerospike provides operations to return a range of elements by rank.
* Rank is 0th-based. 

In [14]:
int secondRank = 1;
int rangeRankSize = 2;

Key key = new Key(listNamespace, listSet, theKey);
Record record = client.get(null, key);
Record rank1And2 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByRankRange(listIntBin, secondRank, rangeRankSize, ListReturnType.VALUE)
    );
        
System.out.println("The integer list: " + record.getValue(listIntBin));
System.out.println("The 2nd and 3rd ranked integers: " + rank1And2.getValue(listIntBin));

The integer list: [81, 3, 27, 9, 27, 1]
The 2nd and 3rd ranked integers: [9, 3]


## UPDATING Lists on the Aerospike Server

Aerospike's [list operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html) can also modify data in the Aerospike database.

### Update the String List in Aerospike

#### Insert a Fish into the Second and Second-from-last Position

In [15]:
String Fish = "Koi";
int secondPosition = 1;
int secondFromLastPosition = -1;

Key key = new Key(listNamespace, listSet, theKey);
Record origRecord = client.get(null, key);
System.out.println("Before – " + origRecord.getValue(listStrBin));

origRecord = client.operate(client.writePolicyDefault, key, 
    ListOperation.insert(listStrBin, secondFromLastPosition, Value.get(Fish)),
    ListOperation.insert(listStrBin, secondPosition, Value.get(Fish))
    );

Record finalRecord = client.get(null, key);
System.out.println(" After – " + finalRecord.getValue(listStrBin));

Before – [Annette, Redwood, Aquamarine, Pineapple]
 After – [Annette, Koi, Redwood, Aquamarine, Koi, Pineapple]


#### Remove By Index from the String List

In [16]:
int firstPosition = 0;

Key key = new Key(listNamespace, listSet, theKey);
Record origRecord = client.get(null, key);
System.out.println("Before – " + origRecord.getValue(listStrBin));

origRecord = client.operate(client.writePolicyDefault, key, 
    ListOperation.remove(listStrBin, firstPosition)
    );

Record finalRecord = client.get(null, key);
System.out.println(" After – " + finalRecord.getValue(listStrBin));

Before – [Annette, Koi, Redwood, Aquamarine, Koi, Pineapple]
 After – [Koi, Redwood, Aquamarine, Koi, Pineapple]


### Update the Integer List in Aerospike

#### Append 17 to the List

In [17]:
int seventeen = 17;

Key key = new Key(listNamespace, listSet, theKey);
Record origRecord = client.get(null, key);
System.out.println("Before – " + origRecord.getValue(listIntBin));

origRecord = client.operate(client.writePolicyDefault, key, 
    ListOperation.append(listIntBin, Value.get(seventeen))
    );

Record finalRecord = client.get(null, key);
System.out.println(" After – " + finalRecord.getValue(listIntBin));

Before – [81, 3, 27, 9, 27, 1]
 After – [81, 3, 27, 9, 27, 1, 17]


#### Increment the 4th Integer by 111

* Indexes into lists start at 0.

In [18]:
int incNum = 111;
int incIndex = 3;

Key key = new Key(listNamespace, listSet, theKey);
Record origRecord = client.get(null, key);
System.out.println("Before – " + origRecord.getValue(listIntBin) );

origRecord = client.operate(client.writePolicyDefault, key, 
    ListOperation.increment(listIntBin, incIndex, Value.get(incNum))
    );

Record finalRecord = client.get(null, key);
System.out.println(" After – " + finalRecord.getValue(listIntBin) );

Before – [81, 3, 27, 9, 27, 1, 17]
 After – [81, 3, 27, 120, 27, 1, 17]


### Sorting the Lists

Aerospike also provides operations to sort lists and optionally remove duplicates.

#### Sort the String and Drop Duplicates

For information on the [flags](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListSortFlags.html) specifying whether to remove duplicates, go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListSortFlags.html). 

In [19]:
import com.aerospike.client.cdt.ListSortFlags;

int dropDuplicates = 2;

Key key = new Key(listNamespace, listSet, theKey);
Record origRecord = client.get(null, key);
System.out.println("Unsorted – " + origRecord.getValue(listStrBin));

origRecord = client.operate(client.writePolicyDefault, key, 
    ListOperation.sort(listStrBin, ListSortFlags.DROP_DUPLICATES)
    );

Record finalRecord = client.get(null, key);
System.out.println("  Sorted – " + finalRecord.getValue(listStrBin));

Unsorted – [Koi, Redwood, Aquamarine, Koi, Pineapple]
  Sorted – [Aquamarine, Koi, Pineapple, Redwood]


####  Sort the Integer List and Keep Duplicates

In [20]:
int keepDuplicates = 0;

Key key = new Key(listNamespace, listSet, theKey);
Record origRecord = client.get(null, key);
System.out.println("Unsorted – " + origRecord.getValue(listIntBin));

origRecord = client.operate(client.writePolicyDefault, key, 
    ListOperation.sort(listIntBin, ListSortFlags.DEFAULT)
    );

Record finalRecord = client.get(null, key);
System.out.println("  Sorted – " + finalRecord.getValue(listIntBin));

Unsorted – [81, 3, 27, 120, 27, 1, 17]
  Sorted – [1, 3, 17, 27, 27, 81, 120]


## DELETING the Records and Closing Server Connection

### Delete the Record from Aerospike

Use the **asinfo** administration tool to drop the index containing our list data.

In [24]:
%sh asinfo -v "truncate:namespace=test;set=listset1;"
System.out.println("Record deleted.");

Record deleted.


### Close the Connection to Aerospike

In [22]:
client.close();
System.out.println("Server connection closed.");

Server connection closed.


## Code Summary

Here is a collection of all of the non-Jupyter code from this tutorial.
1. Import Java Libraries.
2. Import Aerospike Client Libraries.
3. Start the Aerospike Client.
4. Create Test Data.
5. Put Record into Aerospike
6. Get Data from Aerospike.
    1. Get the Record.
    2. Get the Last String and Highest Rank.
    3. Get Integers Between 3 and 27.
    4. Get 2 Integers By Rank Starting with the Second Rank Item. 
7. Update the Record in Aerospike
    1. Add Koi twice to the String List.
    2. Remove the Name from the String List.
    3. Append 17 to the Integer List.
    4. Increment the 4th Integer by 111.
    5. Sort the Strings and Drop Duplicates.
    6. Sort the Integers and Keep Duplicates.
8. Close the Client Connections.

In [23]:
// Import Java Libraries.

import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import java.util.Map;


// Import Aerospike Client Libraries.

import com.aerospike.client.AerospikeClient;
import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Record;
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListReturnType;
import com.aerospike.client.cdt.ListSortFlags;


// Start the Aerospike Client.

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");


// Create Test Data.

ArrayList<String> listStr = new ArrayList<String>();
listStr.add("Annette");
listStr.add("Redwood");
listStr.add("Aquamarine");
listStr.add("Pineapple");
System.out.println("Created String List: " + listStr);

ArrayList<Integer> listInt = new ArrayList<Integer>();
listInt.add(81);
listInt.add(3);
listInt.add(27);
listInt.add(9);
listInt.add(27);
listInt.add(1);
System.out.println("Created Integer List: " + listInt);


// Put Record into Aerospike

String listSet = "listset1";
String listNamespace = "test";
String listStrBin = "liststrbin";
String listIntBin = "listintbin";
ClientPolicy clientPolicy = new ClientPolicy();

Integer theKey = 0;

Key key = new Key(listNamespace, listSet, theKey);
Bin bin1 = new Bin(listStrBin, listStr);
Bin bin2 = new Bin(listIntBin, listInt);
client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
System.out.println("Inserted Key: " + theKey + ", liststrbin: " + listStr + ", listintbin: " + listInt);


// Get Data from Aerospike.
// 1. Get Record.
// 2. Get the Last String and Highest Rank.
// 3. Get Integers Between 3 and 27.
// 4. Get 2 Integers By Rank Starting with the Second Rank Item. 

int last = -1;
int highestRank = -1;
int lowerBound = 3;
int upperBound = 27;
int secondRank = 1;
int rangeRankSize = 2;

Key key = new Key(listNamespace, listSet, theKey);
Record record = client.get(null, key);
Record lastString = client.operate(client.writePolicyDefault, key, 
    ListOperation.get(listStrBin, last)
    );
Record highestRankString = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByRank(listStrBin, highestRank, ListReturnType.VALUE)
    );
Record between3And27 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRange(listIntBin, Value.get(lowerBound), Value.get(upperBound), ListReturnType.VALUE)
    );   

Record rank1And2 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByRankRange(listIntBin, secondRank, rangeRankSize, ListReturnType.VALUE)
    );

System.out.println("Read the Full Record From Aerospike:" + record);
System.out.println("The last string: " + lastString.getValue(listStrBin));
System.out.println("The highest rank string: " + highestRankString.getValue(listStrBin));
System.out.println("The integers between " + lowerBound + " and " + upperBound + ": " + between3And27.getValue(listIntBin));
System.out.println("The 2nd and 3rd ranked integers: " + rank1And2.getValue(listIntBin));


// Update the Record in Aerospike
// 1. Add Koi twice to the String List.
// 2. Remove the Name from the String List.
// 3. Append 17 to the Integer List.
// 4. Increment the 4th Integer by 111.
// 5. Sort the Strings and Drop Duplicates.
// 6. Sort the Integers and Keep Duplicates.

String Fish = "Koi";
int fishIndex0 = 1;
int fishIndex1 = -1;
int nameIndex = 0;
int seventeen = 17;
int incNum = 111;
int incIndex = 3;

origRecord = client.operate(client.writePolicyDefault, key, 
    ListOperation.insert(listStrBin, fishIndex0, Value.get(Fish)),
    ListOperation.insert(listStrBin, fishIndex1, Value.get(Fish)),
    ListOperation.remove(listStrBin, nameIndex),
    ListOperation.append(listIntBin, Value.get(seventeen)),
    ListOperation.increment(listIntBin, incIndex, Value.get(incNum)),
    ListOperation.sort(listStrBin, ListSortFlags.DROP_DUPLICATES),
    ListOperation.sort(listIntBin, ListSortFlags.DEFAULT)
    );

Record finalRecord = client.get(null, key);
System.out.println("After Record Edits – " + finalRecord);
client.close();
System.out.println("Closed client connections.");

Initialized the client and connected to the cluster.
Created String List: [Annette, Redwood, Aquamarine, Pineapple]
Created Integer List: [81, 3, 27, 9, 27, 1]
Inserted Key: 0, liststrbin: [Annette, Redwood, Aquamarine, Pineapple], listintbin: [81, 3, 27, 9, 27, 1]
Read the Full Record From Aerospike:(gen:1),(exp:351830349),(bins:(liststrbin:[Annette, Redwood, Aquamarine, Pineapple]),(listintbin:[81, 3, 27, 9, 27, 1]))
The last string: Pineapple
The highest rank string: Redwood
The integers between 3 and 27: [3, 9]
The 2nd and 3rd ranked integers: [9, 3]
After Record Edits – (gen:2),(exp:351830349),(bins:(liststrbin:[Aquamarine, Koi, Pineapple, Redwood]),(listintbin:[1, 3, 17, 27, 27, 81, 120]))
Closed client connections.


## Aerospike Does Lists

Aerospike database and its Java Client are up to the task of working with your list data.

## What's Next?





### Next Steps

Have questions? Don't hesitate to reach out if you have additional questions about working with lists at https://discuss.aerospike.com/.

Want to check out other Java notebooks?
1. [Hello, World](hello_world.ipynb)
2. [Aerospike Query and UDF](query_udf.ipynb)
3. [Simple Put Get Example](SimplePutGetExample.ipynb)
4. [Working with Twitter Data](tweetaspike.ipynb)

Are you running this from Binder? [Download the Aerospike Notebook Repo](https://github.com/aerospike-examples/interactive-notebooks) and work with Aerospike Database and Jupyter locally using a Docker container.

### Additional Resources

* What else can we do in Java? See the [Aerospike Java Client](https://github.com/aerospike/aerospike-client-java). 
* What other ways can we work with Lists? Take a look at [Aerospike's List Operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html).
* What are Namespaces, Sets, and Bins? Check out the [Aerospike Data Model](https://www.aerospike.com/docs/architecture/data-model.html). 
* How robust is the Aerospike database? Browse the [Aerospike Database Architecture](https://www.aerospike.com/docs/architecture/index.html).